In [2]:
import pandas as pd
import datetime as dt
from sqlalchemy import create_engine

engine = create_engine('sqlite:///../data/rambam.sqlite', echo=False)

# The Visits table

*(Taken from the data dictionary provided with MDB files)*

The visit table includes general information on each patient that enters the hospital on a particular day – each record in this table relates to a specific visit.

1. patient_id – the ID number of the patient
2. medical_id – the medical case number
3. gender – the gender of the patient
4. entry_unit – the hospital unit that the patient arrives at or is admitted to
5. entry_group – indicates the entry group: Emergency Department or directly to Hospital Department
6. first_department – the medical-ward code of the first department to which the patient is admitted
7. first_ward – the medical-ward code of the first ward to which the patient is admitted
8. exit_unit – the hospital unit from which the patient is released
9. exit_group – indicates the exit group: Emergency Department or Hospital Department
10. exit_department – the medical-ward code of the hospital department from which the patient is released
11. outcome – the release type (Released (Home)/ Deceased/ to Medical Clinic/ Renunciation of Treatments /...)
12. entry_date[<sup>*</sup>](#fn1) – the arrival date
13. entry_time[<sup>**</sup>](#fn2) – time in seconds at which the patient arrives at the hospital
14. hospitalization_date[<sup>*</sup>](#fn1) – the hospitalization date
15. hospitalization_time[<sup>**</sup>](#fn2) – time in seconds at which the patient was hospitalized
16. exit_day[<sup>*</sup>](#fn1) – the release date
17. exit_time[<sup>**</sup>](#fn2) – time in seconds at which the patient was released
18. ED_dur – the patient's length of stay in the Emergency department, in seconds.
19. ED_dur_hours – the patient's length of stay in the Emergency department, in hours.
20. hospitalization_dur – the patient's total length of stay in hospital departments, in seconds.
21. hospitalization_dur_hours – the patient's total length of stay in hospital departments, in hours.
22. hospitalization_dur_days – the patient's total length of stay in hospital departments, in days.
23. age[<sup>#</sup>](#fn3) – the patient's age on the arrival date, in years
24. num_dep – the number of departments in which the patient was hospitalized during his/her stay in the hospital


<span id="fn1"> * Date is the number of days since the origin which is time 00:00:00 on 01/01/1970</span><br>
<span id="fn2"> ** Time in seconds is the time since the origin which is time 00:00:00 on 01/01/1970</span><br>
<span id="fn3"> # age of patient , when the patient’s birth date is missing, then the value of the field age is stated to be equal to 999</span><br>

In [10]:
df = pd.read_sql_table('visits', engine, parse_dates=['entry_date', 'hospitalization_date', 'exit_date'])

In [11]:
df.head()

,patient_id,medical_id,gender,entry_unit,entry_group,first_department,first_ward,entry_date,hospitalization_date,exit_date,...,hospitalization_time,exit_day,exit_time,ED_dur,ED_dur_hours,hospitalization_dur,hospitalization_dur_hours,hospitalization_dur_days,age,num_dep
0,10000037,20678055,2,12,2,51,51,2004-04-28 10:16:56,2004-04-28 10:16:56,2004-05-04 12:29:00,...,1083147416,12542,1083673740,0,0,526324,146,6,45,1
1,10000090,20015725,2,1,1,1,39,2004-04-28 23:29:25,2004-04-29 05:30:04,2004-05-13 11:54:21,...,1083216604,12551,1084449261,21639,6,1232657,342,14,65,1
2,10000091,20013872,2,1,1,1,0,2004-04-02 15:11:31,2004-04-02 20:52:52,2004-04-02 20:52:52,...,1080939172,12510,1080939172,20481,5,0,0,0,78,0
3,10000094,20015584,2,1,1,1,46,2004-04-20 11:51:24,2004-04-20 13:21:04,2004-04-28 12:10:59,...,1082467264,12536,1083154259,5380,1,686995,190,7,34,1
4,10000096,20115329,2,1,1,1,39,2004-04-18 17:18:07,2004-04-18 23:59:00,2004-04-20 10:42:30,...,1082332740,12528,1082457750,24053,6,125010,34,1,98,1


# Whitt & Zhang 2017 datasets

Datasets containing the sub sets of data described in:

    Whitt, Ward, and Xiaopei Zhang. ‘A Data-Driven Model of an Emergency Department’. Operations Research for Health Care 12 (1 March 2017): 1–15. https://doi.org/10.1016/j.orhc.2016.11.001.
  
**From the paper:**
* "we focus on the emergency internal medicine unit (EIMU)"
* "We only focus on the 25-week period from December 2004 to May 2005. In particular, we use the 5th, 6th, 13th and 18th columns of the visit table in the database, which are the entry group, first department, entry time and ED duration."
* "The LoS refers to the LoS within the ED up until the time that a decision is made to admit the patient to an IW or not."
* "A total of 58,332 patients visited the comprehensive ED, with 24,317 going to the EIMU (3955, 4360, 3530, 4324, 3965 and 4183 for each month)."

**Constants**

In [12]:
Emergency_Room = 1 # Group code for ER
EIMU = 1 # Unit code of EIMU occurding to the data dictionary

start = (dt.datetime(2004, 12, 1) - dt.datetime(1970, 1, 1)).total_seconds() # Dec 2004
end = (dt.datetime(2005, 6, 1) - dt.datetime(1970, 1, 1)).total_seconds() # May 2005

# "25 week period" as defined in the paper
pstart = (dt.datetime(2004, 12, 5) - dt.datetime(1970, 1, 1)).total_seconds() # Dec 2004
pend = (dt.datetime(2005, 5, 29) - dt.datetime(1970, 1, 1)).total_seconds() # May 2005

# Add non rounded LoS column
df['los_hrs'] = df.ED_dur/3600

# ED exit time
df['ed_exit_time'] = df.entry_time + df.ED_dur

## Dataset 1 - Larger ED

All patients that visited the larger ED during the period.

In [6]:
ds1 = df[
    (df.entry_group == Emergency_Room) &
    (df.entry_time >= start) &
    (df.entry_time < end)
]

assert(len(ds1) == 58332)

## Dataset 2 - ED (EIMU)

>"a subset of dataset 1 containing those patients who went to the EIMU within the comprehensive ED during the 25 week period; we refer to the EIMU simply as the ED."


In [7]:
ds2 = ds1[ds1.first_department == EIMU]

assert(len(ds2) == 24317)

## Dataset 3 - Subset based on 25 week period

> "a subset of dataset 2 containing those patients that arrived within the 25 week period. (We use dataset 2 for occupancy statistics.)"

In [8]:
ds3 = ds2[
    (ds2.entry_time >= pstart) &
    (ds2.entry_time < pend)
]

assert(len(ds3) == 23409)

## Dataset 4 - Subset based on 25 week period

> "a subset of the dataset 2 containing those patients who both entered and departed the system between Dec. 5, 2004, and May 28, 2005, and have LOS less then 48 h."

In [9]:
ds4 = ds2[
    ((ds2.entry_time >= pstart) & (ds2.entry_time < pend)) | 
    ((ds2.ed_exit_time >= pstart)  & (ds2.ed_exit_time < pend))
]
ds4 = ds4[ds4.los_hrs < 48]

assert(len(ds4) == 23421)

## Possible issue

The data dictionary states:

> exit_group – indicates the exit group: Emergency Department or Hospital Department (see Appendix 1: entry_group)

and the definition given in the appendix is:

| code | name           |
| ---- | -------------- |
|  0   | Total          |
|  1   | Emergency Room |
|  2   | Hospital       |

The paper states the following:

> In this paper we use the "exit_group", which we find to be consistent with the "exit_unit", "exit_department" and "num_dep" in the visits table. "exit_group =1" means the patient was released from the Emergency Department and was not admitted to any hospital department; "exit_group=2" means the patient was released from a hospital department, which means that he was admitted to at least one department from the ED. "exit_group=2" means the patient was released from a hospital department, which means that he was admitted to at least one department from the ED.

This appears to be consistent but the description for **Datasets 5, 6, 8 & 9** then appear to missate these definitions. However, the underlying numbers are correct.

## Dataset 5

> "a subset of dataset 3 containing those patients who were admitted after visiting the ED ("exit_group=1")."

**Should read exit_group=2**

In [10]:
ds5 = ds3[ds3.exit_group == 2]

assert(len(ds5) == 9669)

## Dataset 6

> "a subset of dataset 3 containing those patients who were not admitted after visiting the ED ("exit_group=2")."

**Should read exit_group=1**

In [11]:
ds6 = ds3[ds3.exit_group == 1]

assert(len(ds6) == 13740)

## Dataset 7

> "a subset of dataset 2 containing those patients whose departure times are in the 25-week period (from Dec. 5, 2004, to May 28, 2005)."

In [12]:
ds7 = ds2[
    (ds2.ed_exit_time >= pstart)  & (ds2.ed_exit_time < pend)
]

assert(len(ds7) == 23407)

## Dataset 8

> "a subset of dataset 7 containing those patients who were admitted after visiting the ED ("exit_group=1")."

**Should read exit_group=2**

In [13]:
ds8 = ds7[ds7.exit_group == 2]

assert(len(ds8) == 9668)

## Dataset 9

> "a subset of dataset 7 containing those patients who were not admitted after visiting the ED ("exit_group=2")."

**Should read exit_group=1**

In [14]:
ds9 = ds7[ds7.exit_group == 1]


assert(len(ds9) == 13739)

## Save datasets

In [19]:
for i, ds in enumerate([ds1, ds2, ds3, ds4, ds5, ds6, ds7, ds8, ds9], start=1):
    ds.to_csv(f'../data/wz2017_dataset_{i}.csv.gz', index=False, compression="gzip")
    ds.to_parquet(f'../data/wz2017_dataset_{i}.parquet', index=False)

## Recreation of Table 1

To confirm the datasets match, Table 1 from the paper is recreated below.


> *Table 1*: Sample size (number of arrivals) and LoS statistics (in hours) in different views of the ED data.

| No. | Data description    | Sample size       | Mean | stdv | 1st qu. | Median | 3rd qu. |
|-----|---------------------|-------------------|------|------|---------|--------|---------|
| 1   | Larger ED           | 58,332 (182 days) | 3.08 | 5.36 | 1.06    | 2.16   | 3.87    |
| 2   | ED (EIMU)           | 24,317 (182 days) | 4.10 | 3.49 | 1.90    | 3.31   | 5.26    |
| 3   | Truncation of 2     | 23,409 (175 days) | 4.10 | 3.49 | 1.90    | 3.31   | 5.26    |
| 4   | Truncation of 2     | 23,421 (175 days) | 4.09 | 3.41 | 1.90    | 3.31   | 5.26    |
| 5   | Admitted from 3     | 9,669 (175 days)  | 4.01 | 3.26 | 1.68    | 3.45   | 5.53    |
| 6   | Non-admitted from 3 | 13,740 (175 days) | 4.17 | 3.65 | 2.01    | 3.24   | 5.05    |
| 7   | Truncation of 2     | 23,407 (175 days) | 4.10 | 3.50 | 1.90    | 3.31   | 5.27    |
| 8   | Admitted from 7     | 9,668 (175 days)  | 4.01 | 3.27 | 1.68    | 3.45   | 5.53    |
| 9   | Non-admitted from 7 | 13,739 (175 days) | 4.17 | 3.65 | 2.01    | 3.24   | 5.06    |


In [16]:
columns = ["No.", "Data description", "Sample size", "Mean", "stdv", "1st qu.", "Median", "3rd qu."]
descs = ["Larger ED", "ED (EIMU)", "Truncation of 2", "Truncation of 2", "Admitted from 3", "Non-admitted from 3", "Truncation of 2", "Admitted from 7", "Non-admitted from 7"]
rows = []

for i, ds in enumerate([ds1, ds2, ds3, ds4, ds5, ds6, ds7, ds8, ds9]):
    d = ds.los_hrs.describe().round(2)
    rows.append([i+1, descs[i], f'{int(d["count"]):,}'] + list(d[['mean', 'std', '25%', '50%', '75%']]))

tab1 = pd.DataFrame(data=rows, columns=columns).set_index('No.')
tab1

,Data description,Sample size,Mean,stdv,1st qu.,Median,3rd qu.
No.,,,,,,,
1,Larger ED,"58,332",3.08,5.36,1.06,2.16,3.87
2,ED (EIMU),"24,317",4.10,3.49,1.90,3.31,5.26
3,Truncation of 2,"23,409",4.10,3.49,1.90,3.31,5.26
4,Truncation of 2,"23,421",4.10,3.41,1.90,3.31,5.27
5,Admitted from 3,"9,669",4.01,3.26,1.68,3.45,5.53
6,Non-admitted from 3,"13,740",4.17,3.65,2.01,3.24,5.05
7,Truncation of 2,"23,407",4.10,3.50,1.90,3.31,5.27
8,Admitted from 7,"9,668",4.01,3.27,1.68,3.45,5.53
9,Non-admitted from 7,"13,739",4.17,3.65,2.01,3.24,5.06
